# LL84 Processing

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
pd.set_option('display.max_columns', None)

## Step 0: Load in raw data

In [2]:
#load in raw data
# TODO: make sure we get the columns related to on-site FF infra

ll_raw = pd.read_csv('../data/raw_data/DOB/LL84 Energy Efficiency/NYC_Building_Energy_and_Water_Data_Disclosure_for_Local_Law_84__2022-Present__20250915.csv')
schools = gpd.read_file('../data/processed_data/schools/school_points_with_lcgms.geojson')

/tmp/ipykernel_4131/1855729171.py:4: DtypeWarning: Columns (10,16,32,34,42,61,76,77,78,79,80,81,82,91,174,239,240,241,242,243,244,245,246,247,248) have mixed types. Specify dtype option on import or set low_memory=False.
  ll_raw = pd.read_csv('../data/raw_data/DOB/LL84 Energy Efficiency/NYC_Building_Energy_and_Water_Data_Disclosure_for_Local_Law_84__2022-Present__20250915.csv')


## Step 1: Match schools to LL84 data
### We first attempt to match LL84 data to schools using building codes, then BBL (Building-Block-Lot)

In [4]:
# Step zero: filter for K-12 schools
ll84_edu = ll_raw[ll_raw['List of All Property Use Types (GFA) (ft²)'].fillna('').str.contains('|'.join(['K-12 School', 'Pre-school/Daycare', 'Other - Education', 'Worship Facility']), regex=True)]

# Step one: join records building code extracted from regex to "Property Name" field
extracted_building_codes = ll84_edu['Property Name'].str.extractall(r'([LKMQRX]\d{3})')
# For property names with multiple building codes, expand the dataframe so each building code has its own row
ll84_edu_bc = extracted_building_codes.droplevel('match').join(ll84_edu).rename(columns={0:'Building Code'})
# Keep only the most recent, and largest sqft record for each extracted building code 
ll84_edu_bc = ll84_edu_bc.sort_values(['Calendar Year','Largest Property Use Type - Gross Floor Area (ft²)']).drop_duplicates(subset=['Building Code'], keep='last')
schools_matched_on_bc = schools[['Location Name', 'Location Code', 'Building Code', 'Borough Block Lot']].merge(ll84_edu_bc, on='Building Code', how='left', indicator=True).rename(columns={'_merge':'_merge_bc'})
# TODO: The above merge results in a small join explosion for building codes that are associated with multiple properties in the LL84 data. Need to investigate further.
# NOTE: AS: @JW: As this code is written, there are no additional rows that are added to schools_matched_on_bc compared to schools after the merge. Multiple schools do get matched to the same building code, but this is expected behavior. Let me know if I missed something!

In [5]:
# List schools that are still unmatched. Most of these are DOE schools that are new, or charter schools that occupy space in larger, non K-12 buildings.
unmatched_afterbc_schools = schools_matched_on_bc[schools_matched_on_bc['_merge_bc']=='left_only']
print(f'{len(unmatched_afterbc_schools)} schools remain unmatched after building code joins:')
unmatched_afterbc_schools

244 schools remain unmatched after building code joins:


,Location Name,Location Code,Building Code,Borough Block Lot,Calendar Year,Property ID,Property Name,Parent Property ID,Parent Property Name,Year Ending,"NYC Borough, Block and Lot (BBL)",NYC Building Identification Number (BIN),Address 1,City,Postal Code,Primary Property Type - Self Selected,Primary Property Type - Portfolio Manager-Calculated,National Median Reference Property Type,List of All Property Use Types (GFA) (ft²),Largest Property Use Type,Largest Property Use Type - Gross Floor Area (ft²),2nd Largest Property Use Type,2nd Largest Property Use Type - Gross Floor Area (ft²),3rd Largest Property Use Type,3rd Largest Property Use Type - Gross Floor Area (ft²),Year Built,Construction Status,Number of Buildings,Occupancy,Metered Areas (Energy),Metered Areas (Water),ENERGY STAR Score,National Median ENERGY STAR Score,Target ENERGY STAR Score,Reason(s) for No Score,ENERGY STAR Certification - Year(s) Certified (Score),Eligible for Certification for Report PED (Y/N),ENERGY STAR Certification - Last Approval Date,ENERGY STAR Certification - Application Status,Site EUI (kBtu/ft²),Weather Normalized Site EUI (kBtu/ft²),National Median Site EUI (kBtu/ft²),Site Energy Use (kBtu),Weather Normalized Site Energy Use (kBtu),Electricity - Weather Normalized Site Electricity Use (Grid and Onsite Renewables) (kWh),Electricity - Weather Normalized Site Electricity Intensity (Grid and Onsite Renewables) (kWh/ft²),Percent Electricity,Natural Gas - Weather Normalized Site Natural Gas Use (therms),Natural Gas - Weather Normalized Site Natural Gas Intensity (therms/ft²),Source EUI (kBtu/ft²),Weather Normalized Source EUI (kBtu/ft²),National Median Source EUI (kBtu/ft²),Source Energy Use (kBtu),Weather Normalized Source Energy Use (kBtu),Fuel Oil #1 Use (kBtu),Fuel Oil #2 Use (kBtu),Fuel Oil #4 Use (kBtu),Fuel Oil #5 & 6 Use (kBtu),Diesel #2 Use (kBtu),Propane Use (kBtu),Kerosene Use (kBtu),District Steam Use (kBtu),District Hot Water Use (kBtu),District Chilled Water Use (kBtu),Natural Gas Use (kBtu),Natural Gas Use (therms),Electricity Use - Grid Purchase (kBtu),Electricity Use - Grid Purchase (kWh),Electricity Use - Grid Purchase and Generated from Onsite Renewable Systems (kWh),Electricity Use - Grid Purchase and Generated from Onsite Renewable Systems (kBtu),Electricity Use – Generated from Onsite Renewable Systems and Used Onsite (kWh),Electricity Use – Generated from Onsite Renewable Systems and Used Onsite (kBtu),Electricity Use – Generated from Onsite Renewable Systems (kWh),Electricity Use – Generated from Onsite Renewable Systems and Exported (kWh),Annual Maximum Demand (kW),Annual Maximum Demand (MM/YYYY),Annual Maximum Demand (Meter Name (Meter ID)),Green Power - Onsite (kWh),Green Power - Offsite (kWh),Green Power - Onsite and Offsite (kWh),Total (Location-Based) GHG Emissions (Metric Tons CO2e),Total (Location-Based) GHG Emissions Intensity (kgCO2e/ft²),Direct GHG Emissions (Metric Tons CO2e),Direct GHG Emissions Intensity (kgCO2e/ft²),Indirect (Location-Based) GHG Emissions (Metric Tons CO2e),Indirect (Location-Based) GHG Emissions Intensity (kgCO2e/ft²),Net Emissions (Metric Tons CO2e),National Median Total (Location-Based) GHG Emissions (Metric Tons CO2e),Avoided Emissions - Onsite Green Power (Metric Tons CO2e),Avoided Emissions - Offsite Green Power (Metric Tons CO2e),Avoided Emissions - Onsite and Offsite Green Power (Metric Tons CO2e),Percent of Electricity that is Green Power,Percent of RECs Retained,Percent of Total Electricity Generated from Onsite Renewable Systems,LEED US Project ID,Default Values,Temporary Values,Estimated Values - Energy,Estimated Values - Water,Alert - Energy Meter has less than 12 full calendar months of data,Alert - Energy Meter has gaps,Alert - Energy Meter has overlaps,Alert - Energy - No meters selected for metrics,Alert - Energy Meter has single entry more than 65 days,Alert - Property has no uses,Alert - Water Meter has less than 12 full calendar months of data,"Alert - Data Center Issue 

In [6]:
# Step two: for remaining un-joined records, attempt join on BBL
unmatched_bc_schools_loc_codes = schools_matched_on_bc[schools_matched_on_bc['_merge_bc']!='both']['Location Code'].unique()
unmatched_bc_schools = schools[schools['Location Code'].isin(unmatched_bc_schools_loc_codes)].copy()

# LL84 sometimes lists multiple BBLs for a single property. Here, I explode those into one row per BBL to increase chances of a match on BBL.
ll84_edu_bbl = pd.DataFrame(ll84_edu['NYC Borough, Block and Lot (BBL)'].str.split(';', expand=True).stack().droplevel(1)).join(ll84_edu).rename(columns={0:'BBL'})
# Some BBLs have hyphens, so removing those
ll84_edu_bbl['BBL'] = ll84_edu_bbl['BBL'].str.replace('-', '')

In [12]:
ll84_edu_bbl['Largest Property Use Type - Gross Floor Area (ft²)'] = pd.to_numeric(ll84_edu_bbl['Largest Property Use Type - Gross Floor Area (ft²)'], errors='coerce')

In [13]:
#Step three: finally, choose most recent, largest sqft record per BBL
# Drop duplicated BBLs; keep most recent record
# NOTE: @JW I (AS) modified the code below to keep the most recent record per BBL, as well as the largest property. This prevents cases where smaller buildings w/ the same BBL were preserved in the filtering process
# ll84_edu_bbl_bbl = ll84_edu_bbl.sort_values('Report Submission Date').drop_duplicates(subset=['Property ID', 'NYC Borough, Block and Lot (BBL)'], keep='last')
ll84_edu_bbl = ll84_edu_bbl.sort_values(['Calendar Year','Largest Property Use Type - Gross Floor Area (ft²)']).drop_duplicates(subset=['BBL'], keep='last')
schools_matched_on_bbl = unmatched_bc_schools.merge(ll84_edu_bbl, left_on='Borough Block Lot', right_on='BBL', how='left', indicator=True).rename(columns={'_merge':'_merge_bbl'})

In [15]:
# Combine different join results into single DF
ll84_schools_combined = pd.concat([
    schools_matched_on_bc[schools_matched_on_bc['_merge_bc']=='both'],
    schools_matched_on_bbl[schools_matched_on_bbl['_merge_bbl']=='both']
], ignore_index=True)

In [16]:
# List schools that are still unmatched. Most of these are DOE schools that are new, or charter schools that occupy space in larger, non K-12 buildings.
unmatched_afterauto_schools = schools[~schools['Location Code'].isin(ll84_schools_combined['Location Code'].unique())]
print(f'{len(unmatched_afterauto_schools)} schools remain unmatched after Building Code and BBL joins:')
unmatched_afterauto_schools

159 schools remain unmatched after Building Code and BBL joins:


,Location Name,Managed By Name,Location Code,Building Code,ATS,Primary Address,City,State,Zip,Borough Block Lot,Census Tract,Community District,Council District,Latitude,Longitude,BEDS Number,Location Type Description,Location Category Description,Grades,Grades Final,Open Date,NTA,NTA_Name,Principal Name,Principal Title,Principal Phone Number,Fax Number,Geographical District Code,Administrative District Code,Administrative District Location Code,Administrative District Name,Community School Sup Name,HighSchool Network Location Code,HighSchool Network Name,HighSchool Network Superintendent,HighSchool Network Superintendent Email,BCO Location Code,in_LCGMS,full_address,google_lat,google_lng,google_location_type,lat,lng,geometry
264,The Detective WenJian Liu School of Civics and...,DOE,K331,K676,20K331,6312 13 AVENUE,BROOKLYN,NY,11219,3057390002,194,310,43,40.627038,-74.000890,332000010331,General Academic,Elementary,"0K,01,SE","0K,01,02,03,04,05,SE",Jul 1 2024,BK30,Dyker Heights,Lisa Lin,Principal,None,None,20,20,K820,COMMUNITY SCHOOL DISTRICT 20,"PRETTO, DAVID",None,None,None,None,KFSS,True,"6312 13 AVENUE, BROOKLYN, NY 11219",NaN,NaN,None,40.627038,-74.000890,POINT (-74.00089 40.62704)
286,Hellenic Classical Charter School,Charter,K362,KAWR,84K362,646 FIFTH AVENUE,BROOKLYN,NY,11215,3006340034,14300,307,38,40.662985,-73.992254,331500860878,General Academic,K-8,"PK,0K,01,02,03,04,05,06,07,08","0K,01,02,03,04,05,06,07,08",Jul 1 2005,BK37,Park Slope-Gowanus,Christina Tettonis/Natasha Caban,Principal,718-499-0957,718-499-0959,15,84,None,OFFICE OF CHARTER SCHOOLS,None,None,None,None,None,None,True,"646 FIFTH AVENUE, BROOKLYN, NY 11215",NaN,NaN,None,40.662985,-73.992254,POINT (-73.99225 40.66298)
317,"School of Technology, Arts, and Research",DOE,K407,K322,20K407,650 86 STREET,BROOKLYN,NY,11228,3060560015,152,310,43,40.618628,-74.022073,332000010407,General Academic,Junior High-Intermediate-Middle,"06,07,SE","06,07,08,SE",Jul 1 2024,BK30,Dyker Heights,Stephen Lombardo,Principal,None,None,20,20,K820,COMMUNITY SCHOOL DISTRICT 20,"PRETTO, DAVID",None,None,None,None,KFSS,True,"650 86 STREET, BROOKLYN, NY 11228",NaN,NaN,None,40.618628,-74.022073,POINT (-74.02207 40.61863)
322,Joanne Seminara School of Law and Medicine,DOE,K413,K053,20K413,280 SENATOR STREET,BROOKLN,NY,11220,3058530045,70,310,47,40.637031,-74.025888,332000010413,General Academic,Elementary,"0K,01,SE","0K,01,02,03,04,05,SE",Jul 1 2024,BK31,Bay Ridge,Saher Said,Principal,None,None,20,20,K820,COMMUNITY SCHOOL DISTRICT 20,"PRETTO, DAVID",None,None,None,None,KFSS,True,"280 SENATOR STREET, BROOKLN, NY 11220",NaN,NaN,None,40.637031,-74.025888,POINT (-74.02589 40.63703)
349,Williamsburg Charter High School,Charter,K473,KBYQ,84K473,198 VARET STREET,BROOKLYN,NY,11206,0,48500,301,34,40.703703,-73.935969,331400860865,General Academic,High school,"09,10,11,12","09,10,11,12",Aug 31 2004,BK78,Bushwick South,Valerie Jacobson,Principal,718-782-9830,347-464-7604,14,84,None,OFFICE OF CHARTER SCHOOLS,None,None,None,None,None,None,True,"198 VARET STREET, BROOKLYN, NY 11206",NaN,NaN,None,40.703703,-73.935969,POINT (-73.93597 40.7037)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1952,Harriet Tubman Charter School,Charter,X706,X256,84X706,3565 THIRD AVENUE,BRONX,NY,10456,2023730014,14500,203,16,40.832328,-73.905771,320900860823,General Academic,K-8,"0K,01,02,03,04,05,06,07,08","0K,01,02,03,04,05,06,07,08",Jul 1 2000,BX01,Claremont-Bathgate,Cleveland Person,Principal,718-537-9912,718-537-9858,9,84,None,OFFICE OF CHARTER SCHOOLS,None,None,None,None,None,None,True,"3565 THIRD AVENUE, BRONX, NY 10456",NaN,NaN,None,40.832328,-73.905771,POINT (-73.90577 40.83233)
1957,Bronx Charter School for the Arts,Charter,X730,XAFM,84X730,950 LONGFELLOW AVENUE,BRONX,NY,10474,2027550125,11502,202,17,40.821815,-73.886477,320800860846,General Academic,K-8,"0K,01,02,03,04,05,06,07,0

### Manual matching to correct for missed matches

In [37]:
#NOTE @JW manual matches (Location Code, to the row in the ll84 data) for remaining unmatched schools, that did not pass the building code or BBL extractions
#commented out matches have building code, not the location code
manual_matches = {
    #'K241': 36863,
    'Q687': 47929,
    'M085': 5179,
    #'X445': 36862,
    #'K811' : 52066,
    #'K092' : 38626,
    #'K490' : 52758,
    #'M705' : 49040,
    #'M204' : 46251,
    #'M284' : 34331,
    #'K535' : 52810,
    #'K312' : 36864,
    #'Q136' : 36867,
    #'X614' : 57812,
    #'X177' : 51985,
    #'X063' : 51984,
    #'X008' : 52538,
    #'X310' : 52376,
    #'X226' : 39016    
}

manual_add = ll_raw.loc[manual_matches.values()]
manual_add['Location Code'] = manual_matches.keys()
manual_add['Building Code'] = schools[schools['Location Code'].isin(manual_add['Location Code'])]['Building Code'].to_list()
ll84_matched_bc = pd.concat([ll84_schools_combined, manual_add])

In [38]:
# List schools that are still unmatched. Most of these are DOE schools that are new, or charter schools that occupy space in larger, non K-12 buildings.
unmatched_aftermanual_schools = schools[~schools['Location Code'].isin(ll84_matched_bc['Location Code'])]
print(f'{len(unmatched_aftermanual_schools)} schools remain unmatched after building code and BBL joins:')
unmatched_aftermanual_schools

157 schools remain unmatched after building code and BBL joins:


,Location Name,Managed By Name,Location Code,Building Code,ATS,Primary Address,City,State,Zip,Borough Block Lot,Census Tract,Community District,Council District,Latitude,Longitude,BEDS Number,Location Type Description,Location Category Description,Grades,Grades Final,Open Date,NTA,NTA_Name,Principal Name,Principal Title,Principal Phone Number,Fax Number,Geographical District Code,Administrative District Code,Administrative District Location Code,Administrative District Name,Community School Sup Name,HighSchool Network Location Code,HighSchool Network Name,HighSchool Network Superintendent,HighSchool Network Superintendent Email,BCO Location Code,in_LCGMS,full_address,google_lat,google_lng,google_location_type,lat,lng,geometry
264,The Detective WenJian Liu School of Civics and...,DOE,K331,K676,20K331,6312 13 AVENUE,BROOKLYN,NY,11219,3057390002,194,310,43,40.627038,-74.000890,332000010331,General Academic,Elementary,"0K,01,SE","0K,01,02,03,04,05,SE",Jul 1 2024,BK30,Dyker Heights,Lisa Lin,Principal,None,None,20,20,K820,COMMUNITY SCHOOL DISTRICT 20,"PRETTO, DAVID",None,None,None,None,KFSS,True,"6312 13 AVENUE, BROOKLYN, NY 11219",NaN,NaN,None,40.627038,-74.000890,POINT (-74.00089 40.62704)
286,Hellenic Classical Charter School,Charter,K362,KAWR,84K362,646 FIFTH AVENUE,BROOKLYN,NY,11215,3006340034,14300,307,38,40.662985,-73.992254,331500860878,General Academic,K-8,"PK,0K,01,02,03,04,05,06,07,08","0K,01,02,03,04,05,06,07,08",Jul 1 2005,BK37,Park Slope-Gowanus,Christina Tettonis/Natasha Caban,Principal,718-499-0957,718-499-0959,15,84,None,OFFICE OF CHARTER SCHOOLS,None,None,None,None,None,None,True,"646 FIFTH AVENUE, BROOKLYN, NY 11215",NaN,NaN,None,40.662985,-73.992254,POINT (-73.99225 40.66298)
317,"School of Technology, Arts, and Research",DOE,K407,K322,20K407,650 86 STREET,BROOKLYN,NY,11228,3060560015,152,310,43,40.618628,-74.022073,332000010407,General Academic,Junior High-Intermediate-Middle,"06,07,SE","06,07,08,SE",Jul 1 2024,BK30,Dyker Heights,Stephen Lombardo,Principal,None,None,20,20,K820,COMMUNITY SCHOOL DISTRICT 20,"PRETTO, DAVID",None,None,None,None,KFSS,True,"650 86 STREET, BROOKLYN, NY 11228",NaN,NaN,None,40.618628,-74.022073,POINT (-74.02207 40.61863)
322,Joanne Seminara School of Law and Medicine,DOE,K413,K053,20K413,280 SENATOR STREET,BROOKLN,NY,11220,3058530045,70,310,47,40.637031,-74.025888,332000010413,General Academic,Elementary,"0K,01,SE","0K,01,02,03,04,05,SE",Jul 1 2024,BK31,Bay Ridge,Saher Said,Principal,None,None,20,20,K820,COMMUNITY SCHOOL DISTRICT 20,"PRETTO, DAVID",None,None,None,None,KFSS,True,"280 SENATOR STREET, BROOKLN, NY 11220",NaN,NaN,None,40.637031,-74.025888,POINT (-74.02589 40.63703)
349,Williamsburg Charter High School,Charter,K473,KBYQ,84K473,198 VARET STREET,BROOKLYN,NY,11206,0,48500,301,34,40.703703,-73.935969,331400860865,General Academic,High school,"09,10,11,12","09,10,11,12",Aug 31 2004,BK78,Bushwick South,Valerie Jacobson,Principal,718-782-9830,347-464-7604,14,84,None,OFFICE OF CHARTER SCHOOLS,None,None,None,None,None,None,True,"198 VARET STREET, BROOKLYN, NY 11206",NaN,NaN,None,40.703703,-73.935969,POINT (-73.93597 40.7037)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1952,Harriet Tubman Charter School,Charter,X706,X256,84X706,3565 THIRD AVENUE,BRONX,NY,10456,2023730014,14500,203,16,40.832328,-73.905771,320900860823,General Academic,K-8,"0K,01,02,03,04,05,06,07,08","0K,01,02,03,04,05,06,07,08",Jul 1 2000,BX01,Claremont-Bathgate,Cleveland Person,Principal,718-537-9912,718-537-9858,9,84,None,OFFICE OF CHARTER SCHOOLS,None,None,None,None,None,None,True,"3565 THIRD AVENUE, BRONX, NY 10456",NaN,NaN,None,40.832328,-73.905771,POINT (-73.90577 40.83233)
1957,Bronx Charter School for the Arts,Charter,X730,XAFM,84X730,950 LONGFELLOW AVENUE,BRONX,NY,10474,2027550125,11502,202,17,40.821815,-73.886477,320800860846,General Academic,K-8,"0K,01,02,03,04,05,06,07,0

## Step 3: Clean and save LL84 data, merged with `Location Code` and `Building Code`.

In [39]:
data_columns = [
           'ENERGY STAR Score',
           'Site EUI (kBtu/ft²)',
           'Site Energy Use (kBtu)',
           'Percent Electricity',
           'Direct GHG Emissions (Metric Tons CO2e)',
           'Direct GHG Emissions Intensity (kgCO2e/ft²)',
           'Water Use (All Water Sources) (kgal)',          
           'Weather Normalized Site EUI (kBtu/ft²)',
           'Weather Normalized Site Energy Use (kBtu)',
           'Fuel Oil #1 Use (kBtu)',
           'Fuel Oil #2 Use (kBtu)',
           'Fuel Oil #4 Use (kBtu)',
           'Fuel Oil #5 & 6 Use (kBtu)',
           'Diesel #2 Use (kBtu)',
           'Propane Use (kBtu)',
           'Kerosene Use (kBtu)',
           'District Steam Use (kBtu)',
           'District Hot Water Use (kBtu)',
           'District Chilled Water Use (kBtu)',
           'Natural Gas Use (kBtu)',
           'Electricity Use - Grid Purchase (kBtu)',
           'Electricity Use - Grid Purchase (kWh)',
           'Electricity Use – Generated from Onsite Renewable Systems (kWh)',
           'Electricity Use – Generated from Onsite Renewable Systems and Exported (kWh)',
           'Green Power - Onsite (kWh)',
           'Avoided Emissions - Onsite Green Power (Metric Tons CO2e)',
           'Percent of Electricity that is Green Power',
           'Percent of Total Electricity Generated from Onsite Renewable Systems',
           'Report Submission Date']



ll84_matched_bc[data_columns[:-1]] = ll84_matched_bc[data_columns[:-1]].apply(pd.to_numeric, errors='coerce')

merged_geo = schools[['Location Code', 'Building Code', 'geometry']].merge(ll84_matched_bc[['Building Code'] + data_columns]
                       .drop_duplicates(subset='Building Code', keep='first'), on="Building Code", how='left')

In [41]:
#save final data
merged_geo.to_file('../data/processed_data/energy_water/ll84.geojson', driver='GeoJSON')